
<center>ESTATÍSTICA  APLICADA  À COMPUTAÇÃO</center>
<center>PROJETO FINAL</center>
<center>ALUNOS: Larissa Maciel Belarmino de Melo e Victor Hugo de Oliveira Gangorra</center>


------
## Neste projeto foi criado um Dashboard para visualização de indicadores de Segurança Pública no Brasil.

- O objeto é mostrar a quantidade de ocorrências e vítimas de casos criminais resistradas por mês em cada estado brasileiro a partir do ano de 2015 ao primeiro trimestre do ano de 2022.

In [1]:
!pip install dash dash-bootstrap-components plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 12.5 MB/s eta 0:00:00


In [2]:
#Importação das bibliotecas que serão utilizadas para o desevolvimento do Dash.

from dash import Dash, html, dcc, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

- OBS.: Antes de importar cada biblioteca, efetuei a instalação no Anaconda Prompt com pip install das bibliotecas que não tinham no pacote.

- A seguir, leio os aquivos de dados que serão utilizados no projeto e crio um DataFrame. Foram efetuadas algumas modificações no DataFrame para melhorar a exibição dos dados.

In [3]:
df_ocorrencias = pd.read_excel('Ocorrências.xlsx')
df_ocorrencias.rename(columns = {'Tipo Crime':'Tipo_Crime'}, inplace=True)
df_ocorrencias

,UF,Tipo_Crime,Ano,Mês,Ocorrências
0,Acre,Estupro,2022,janeiro,31
1,Acre,Furto de veículo,2022,janeiro,50
2,Acre,Homicídio doloso,2022,janeiro,10
3,Acre,Lesão corporal seguida de morte,2022,janeiro,0
4,Acre,Roubo a instituição financeira,2022,janeiro,0
...,...,...,...,...,...
20952,Tocantins,Roubo a instituição financeira,2015,dezembro,6
20953,Tocantins,Roubo de carga,2015,dezembro,1
20954,Tocantins,Roubo de veículo,2015,dezembro,55
20955,Tocantins,Roubo seguido de morte (latrocínio),2015,dezembro,2


In [4]:
df_vitimas = pd.read_excel('Crimes.xlsx')
df_vitimas.rename(columns = {'Tipo Crime':'Tipo_Crime', 'Sexo da Vítima':'Sexo_Vítima'}, inplace=True)
df_vitimas.drop(df_vitimas.loc[df_vitimas['Sexo_Vítima']=='Sexo NI'].index, inplace=True)
df_vitimas

,UF,Tipo_Crime,Ano,Mês,Sexo_Vítima,Vítimas
0,Acre,Homicídio doloso,2022,janeiro,Feminino,2
1,Acre,Homicídio doloso,2022,janeiro,Masculino,8
3,Acre,Homicídio doloso,2022,fevereiro,Feminino,0
4,Acre,Homicídio doloso,2022,fevereiro,Masculino,10
6,Acre,Homicídio doloso,2022,março,Feminino,2
...,...,...,...,...,...,...
17927,Tocantins,Roubo seguido de morte (latrocínio),2015,junho,Masculino,1
17928,Tocantins,Roubo seguido de morte (latrocínio),2015,julho,Masculino,1
17929,Tocantins,Roubo seguido de morte (latrocínio),2015,agosto,Masculino,1
17930,Tocantins,Roubo seguido de morte (latrocínio),2015,outubro,Masculino,2


In [5]:
df=pd.concat([df_ocorrencias, df_vitimas])
df = df.fillna(0)
df.Ocorrências = df.Ocorrências.astype(int)
df.Vítimas = df.Vítimas.astype(int)
df = df.drop(columns=['Sexo_Vítima'])
df

,UF,Tipo_Crime,Ano,Mês,Ocorrências,Vítimas
0,Acre,Estupro,2022,janeiro,31,0
1,Acre,Furto de veículo,2022,janeiro,50,0
2,Acre,Homicídio doloso,2022,janeiro,10,0
3,Acre,Lesão corporal seguida de morte,2022,janeiro,0,0
4,Acre,Roubo a instituição financeira,2022,janeiro,0,0
...,...,...,...,...,...,...
17927,Tocantins,Roubo seguido de morte (latrocínio),2015,junho,0,1
17928,Tocantins,Roubo seguido de morte (latrocínio),2015,julho,0,1
17929,Tocantins,Roubo seguido de morte (latrocínio),2015,agosto,0,1
17930,Tocantins,Roubo seguido de morte (latrocínio),2015,outubro,0,2


In [6]:
#Listas de Opções:

Opc_Estados=list(df.UF.unique())

Opc_Crime=list(df.Tipo_Crime.unique())

Opc_Mês=list(df.Mês.unique())

Opc_Ano=list(df.Ano.unique())

- Aqui onde a "mágica" acontece, utilizando `HTML` e outras bibliotecas, foi feito todo o layout do Dash como todos os textos, os gráficos e botões de seleção que vai conter nele. A partir disso, é feita a interação dos botões de opções com os gráficos que serão executados. Esses gráficos foram criados através de uma função que vai receber as opções e implementar os respectivos dados nos gráficos. Por fim, usamos o comando `run_server(debug=False)` para executar e visualizar o Dash.

In [7]:
# Inicializando o app Dash
app = Dash(__name__)

# Layout do app
app.layout = html.Div([
    html.H1("Indicadores de Segurança Pública", style={'textAlign': 'center'}),

    # Dropdown para selecionar o ano
    html.Div([
        html.P("Selecione o Ano:"),
        dcc.Dropdown(
            id='dropdown-ano',
            options=[{'label': str(ano), 'value': ano} for ano in df['Ano'].unique()],
            value=df['Ano'].min()  # Valor inicial
        )
    ], style={'width': '48%', 'display': 'inline-block'}),

    # Dropdown para selecionar o estado
    html.Div([
        html.P("Selecione o Estado:"),
        dcc.Dropdown(
            id='dropdown-estado',
            options=[{'label': estado, 'value': estado} for estado in df['UF'].unique()],
            value=df['UF'].unique()[0]  # Valor inicial
        )
    ], style={'width': '48%', 'display': 'inline-block', 'float': 'right'}),

    # Gráfico 1: Ocorrências por Crime
    html.Div([
        dcc.Graph(id='grafico-ocorrencias')
    ]),

    # Gráfico 2: Vítimas por Crime
    html.Div([
        dcc.Graph(id='grafico-vitimas')
    ])
])

# Callback para atualizar os gráficos com base no ano e estado selecionados
@app.callback(
    [Output('grafico-ocorrencias', 'figure'),
     Output('grafico-vitimas', 'figure')],
    [Input('dropdown-ano', 'value'),
     Input('dropdown-estado', 'value')]
)
def atualizar_graficos(ano_selecionado, estado_selecionado):
    # Filtrar os dados com base no ano e estado selecionados
    df_filtrado = df[(df['Ano'] == ano_selecionado) & (df['UF'] == estado_selecionado)]

    # Gráfico 1: Ocorrências por Crime
    grafico_ocorrencias = px.histogram(
        df_filtrado, x='Tipo_Crime', y='Ocorrências', color='Mês',
        labels={'Tipo_Crime': 'Tipo de Crime', 'Ocorrências': 'Nº de Ocorrências'},
        title=f'Ocorrências por Crime ({estado_selecionado}, {ano_selecionado})'
    )

    # Gráfico 2: Vítimas por Crime
    grafico_vitimas = px.histogram(
        df_filtrado, x='Tipo_Crime', y='Vítimas', color='Mês',
        labels={'Tipo_Crime': 'Tipo de Crime', 'Vítimas': 'Nº de Vítimas'},
        title=f'Vítimas por Crime ({estado_selecionado}, {ano_selecionado})'
    )

    return grafico_ocorrencias, grafico_vitimas

# Executando o app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>